In [1]:
import numpy as np
from datetime import datetime, timedelta
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import metpy
from pyproj import Proj

In [2]:
def rounded_to_the_last_30_minute():
    now = datetime.now()
    rounded = now - (now - datetime.min) % timedelta(minutes=30)
    return rounded

In [3]:
date = rounded_to_the_last_30_minute()
date

datetime.datetime(2023, 10, 30, 2, 30)

In [4]:
YYYYMMDD_HHMM = date.strftime('%Y%m%d_%H%M')
YYYYMMDD_HHMM

'20231030_0230'

In [5]:
File = "https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/NDFD/NWS/CONUS/CONDUIT/NDFD_NWS_CONUS_conduit_2p5km_"+YYYYMMDD_HHMM+".grib2"
File

'https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/NDFD/NWS/CONUS/CONDUIT/NDFD_NWS_CONUS_conduit_2p5km_20231030_0230.grib2'

In [6]:
ds = xr.open_dataset(File)
#ds

In [7]:
ds = ds.metpy.parse_cf()
ds = ds.metpy.assign_latitude_longitude(force=False)
ds

<xarray.Dataset>
Dimensions:                                                                   (
                                                                               time: 6,
                                                                               time_bounds_1: 2,
                                                                               time1: 12,
                                                                               time1_bounds_1: 2,
                                                                               time3: 3,
                                                                               ...
                                                                               time4_bounds_1: 2,
                                                                               x: 2145,
                                                                               y: 1377,
                                                                               time2: 48,
                                                                               height_above_ground1: 1,
                                                                               height_above_ground: 1)
Coordinates: (12/13)
    reftime                                                                   datetime64[ns] ...
  * time                                                                      (time) datetime64[ns] ...
  * time1                                                                     (time1) datetime64[ns] ...
  * time3                                                                     (time3) datetime64[ns] ...
  * time4                                                                     (time4) datetime64[ns] ...
  * x                                                                         (x) float32 ...
    ...                                                                        ...
  * time2                                                                     (time2) datetime64[ns] ...
  * height_above_ground1                                                      (height_above_ground1) float32 ...
    metpy_crs                                                                 object ...
  * height_above_ground                                                       (height_above_ground) float32 ...
    latitude                                                                  (y, x) float64 ...
    longitude                                                                 (y, x) float64 ...
Dimensions without coordinates: time_bounds_1, time1_bounds_1, time3_bounds_1,
                                time4_bounds_1
Data variables: (12/20)
    LambertConformal_Projection                                               int32 ...
    time_bounds                                                               (time, time_bounds_1) datetime64[ns] ...
    time1_bounds                                                              (time1, time1_bounds_1) datetime64[ns] ...
    time3_bounds                                                              (time3, time3_bounds_1) datetime64[ns] ...
    time4_bounds                                                              (time4, time4_bounds_1) datetime64[ns] ...
    Dewpoint_temperature_height_above_ground                                  (time2, height_above_ground1, y, x) float32 ...
    ...                                                                        ...
    Total_snowfall_surface_6_Hour_Accumulation                                (time1, y, x) float32 ...
    Apparent_Temperature_height_above_ground                                  (time2, height_above_ground1, y, x) float32 ...
    Weather_string_surface                                                    (time2, y, x) float32 ...
    Wind_direction_from_which_blowing_height_above_ground                     (time2, height_above_ground, y, x) float32 ...
    Wind_speed_height_above_ground                                      

In [8]:
x, y = ds.x, ds.y

In [9]:
max_temp = ds.Maximum_temperature_height_above_ground_12_Hour_Maximum

In [10]:
proj_data = max_temp.metpy.cartopy_crs
proj_data;

In [11]:
pFull = Proj(proj_data)

In [12]:
def find_closest(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx

In [13]:
siteName = "ETEC"
siteLat, siteLon = (42.68, -73.81) #lat & lon of gridpoint over ETEC
siteX, siteY = pFull(siteLon, siteLat)
siteXidx, siteYidx = find_closest(x, siteX), find_closest(y, siteY)

In [14]:
forecastMax = max_temp.isel(x = siteXidx, y = siteYidx).isel()
forecastMax

<xarray.DataArray 'Maximum_temperature_height_above_ground_12_Hour_Maximum' (
                                                                             time3: 3,
                                                                             height_above_ground1: 1)>
[3 values with dtype=float32]
Coordinates:
    reftime               datetime64[ns] 2023-10-30T02:30:00
  * time3                 (time3) datetime64[ns] 2023-10-30T23:30:00 ... 2023...
    x                     float32 1.816e+06
    y                     float32 2.141e+06
  * height_above_ground1  (height_above_ground1) float32 2.0
    metpy_crs             object Projection: lambert_conformal_conic
    latitude              float64 42.68
    longitude             float64 -73.81
Attributes: (12/14)
    long_name:                       Maximum temperature (12_Hour Maximum) @ ...
    units:                           K
    description:                     Maximum temperature
    grid_mapping:                    LambertConformal_Projection
    Grib_Statistical_Interval_Type:  Maximum
    Grib_Variable_Id:                VAR_0-0-4_L103_I12_Hour_S2
    ...                              ...
    Grib2_Parameter_Category:        Temperature
    Grib2_Parameter_Name:            Maximum temperature
    Grib2_Level_Type:                103
    Grib2_Level_Desc:                Specified height level above ground
    Grib2_Generating_Process_Type:   Forecast
    Grib2_Statistical_Process_Type:  Maximum

In [15]:
timeDimMax, vertDimMax = forecastMax.metpy.time.name, forecastMax.metpy.vertical.name
timeDimMax, vertDimMax

('time3', 'height_above_ground1')

In [16]:
idxTimeTemp = slice(None, 2) # First time
idxVertTemp = 0 # First (and in this case, only) vertical level

timeDictMax = {timeDimMax: idxTimeTemp}
vertDictMax = {vertDimMax: idxVertTemp}

timeDictMax, vertDictMax

({'time3': slice(None, 2, None)}, {'height_above_ground1': 0})

In [17]:
forecastMax = forecastMax.isel(vertDictMax).isel(timeDictMax)
forecastMax = forecastMax.metpy.convert_units('degF')
forecastMax

Magnitude,[47.02999496459961 45.94998550415039]
Units,degree_Fahrenheit


In [18]:
min_temp = ds.Minimum_temperature_height_above_ground_12_Hour_Minimum

In [19]:
forecastMin = min_temp.isel(x = siteXidx, y = siteYidx).isel()

In [20]:
timeDimMin, vertDimMin = forecastMin.metpy.time.name, forecastMin.metpy.vertical.name
timeDimMin, vertDimMin

('time4', 'height_above_ground1')

In [21]:
timeDictMin = {timeDimMin: idxTimeTemp}
vertDictMin = {vertDimMin: idxVertTemp}

timeDictMin, vertDictMin

({'time4': slice(None, 2, None)}, {'height_above_ground1': 0})

In [22]:
forecastMin = forecastMin.isel(vertDictMin).isel(timeDictMin)
forecastMin = forecastMin.metpy.convert_units('degF')
forecastMin

Magnitude,[41.99001693725586 34.069976806640625]
Units,degree_Fahrenheit


In [23]:
precip = ds.Total_precipitation_surface_12_Hour_Accumulation_probability_above_0p254

In [24]:
forecastPrecip = precip.isel(x = siteXidx, y = siteYidx).isel()

In [25]:
timeDimPrecip = forecastPrecip.metpy.time.name

In [26]:
idxTimeFull = slice(None, 4) # First 4 times

timeDictPrecip = {timeDimPrecip: idxTimeFull}

timeDictPrecip

{'time': slice(None, 4, None)}

In [27]:
forecastPrecip = forecastPrecip.isel(timeDictPrecip)
forecastPrecip

<xarray.DataArray 'Total_precipitation_surface_12_Hour_Accumulation_probability_above_0p254' (
                                                                                              time: 4)>
[4 values with dtype=float32]
Coordinates:
    reftime    datetime64[ns] 2023-10-30T02:30:00
  * time       (time) datetime64[ns] 2023-10-30T11:30:00 ... 2023-10-31T23:30:00
    x          float32 1.816e+06
    y          float32 2.141e+06
    metpy_crs  object Projection: lambert_conformal_conic
    latitude   float64 42.68
    longitude  float64 -73.81
Attributes: (12/16)
    long_name:                       Probability Total precipitation (12_Hour...
    units:                           %
    description:                     Total precipitation
    grid_mapping:                    LambertConformal_Projection
    Grib_Statistical_Interval_Type:  Accumulation
    Grib_Variable_Id:                VAR_0-1-8_L1_I12_Hour_S1_Prob_above_0p254
    ...                              ...
    Grib2_Level_Type:                1
    Grib2_Level_Desc:                Ground or water surface
    Grib2_Probability_Type:          1
    Grib2_Probability_Name:          above_0.254
    Grib2_Generating_Process_Type:   Forecast
    Grib2_Statistical_Process_Type:  Accumulation

In [28]:
times = forecastPrecip.metpy.time.values
times

array(['2023-10-30T11:30:00.000000000', '2023-10-30T23:30:00.000000000',
       '2023-10-31T11:30:00.000000000', '2023-10-31T23:30:00.000000000'],
      dtype='datetime64[ns]')

In [29]:
i=0
imax=0
imin=0

print('----------------')

for time_step in times:
    
    period = i+1
    periodStr = str(period)
    
    print('Forecast Period: '+periodStr)
    
    timeStr = pd.to_datetime(str(time_step)) 
    timeStr = timeStr.strftime('%Y-%m-%d %H%M UTC')
    
    print('Valid through: '+timeStr)
    
    timeDictMax = {timeDimMax: imax}
    timeDictMin = {timeDimMin: imin}
    forecastMaxNew = forecastMax.isel(timeDictMax)
    forecastMinNew = forecastMin.isel(timeDictMin)
    
    if forecastMaxNew.metpy.time.values == time_step:
        roundedMax = round(forecastMaxNew.item(0), 1)
        maxStr = str(roundedMax)
        print('Forecast High: '+maxStr)
        if imax < 1:
            imax = imax+1
    
    elif forecastMinNew.metpy.time.values  == time_step:
        roundedMin = round(forecastMinNew.item(0), 1)
        minStr = str(roundedMin)
        print('Forecast Low: '+minStr)
        if imin < 1:
            imin = imin+1
    
    timeDictPrecip = {timeDimPrecip: i}
    forecastPrecipNew = forecastPrecip.isel(timeDictPrecip)
    precipStr = str(forecastPrecipNew.item(0))
    print('Probability of Measurable Precip: '+precipStr+'%')
    
    print('----------------')
    
    i=i+1  

----------------
Forecast Period: 1
Valid through: 2023-10-30 1130 UTC
Forecast Low: 42.0 degree_Fahrenheit
Probability of Measurable Precip: 90.0%
----------------
Forecast Period: 2
Valid through: 2023-10-30 2330 UTC
Forecast High: 47.0 degree_Fahrenheit
Probability of Measurable Precip: 90.0%
----------------
Forecast Period: 3
Valid through: 2023-10-31 1130 UTC
Forecast Low: 34.1 degree_Fahrenheit
Probability of Measurable Precip: 8.0%
----------------
Forecast Period: 4
Valid through: 2023-10-31 2330 UTC
Forecast High: 45.9 degree_Fahrenheit
Probability of Measurable Precip: 5.0%
----------------
